In [1]:
import matplotlib.pyplot as plt

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline

import os
from utils import *

import os

from utils_calcium import ProcessCalcium

Autosaving every 180 seconds
Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
####################################################
################## LOAD DATA #######################
####################################################

# TODOS:
# 1.   load all masks and look at them
#      see if we can find the correct cell in each day - see how much drift there is
# 2.   load all traces and compute reward triggered averages 
#     - this should help understand if other cells in the networks are acquiring a relationship with the reward and the ensmeble cells


root_dir = '/media/cat/8TB/donato/bmi/' #DON-011733/20230302/plane0/merged/F.npy

# M1 mice
M1 = [
    
    'DON-011733',
    'DON-014382',
    'DON-014451',
    'DON-014575',
    'DON-014618',
    'DON-015821',
    
    ]

# CA3 mice
CA3 = [
    'DON-012502',
    'DON-014266',
    'DON-014371',
    'DON-015801',

]
########################################

#
animal_id = 'DON-011733'

# this loads data from the merged folder
c = ProcessCalcium(root_dir,
                   animal_id)

#
c.remove_bad_cells = False

#
c.recompute_binarization = False  # IF TRue: must input correct params from bin notebooks
c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
c.percentile_threshold = 0.99   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
c.maximum_std_of_signal = 0.25     # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it

#
c.save_figures = False
c.load_sessions()

#
print ("...DONE...")

  0%|          | 0/8 [00:00<?, ?it/s]

 50%|█████     | 4/8 [00:21<00:21,  5.33s/it]/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:491: RuntimeWarning: invalid value encountered in divide
  cumsum = cumsum/np.max(cumsum)
 62%|██████▎   | 5/8 [00:26<00:16,  5.34s/it]/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:491: RuntimeWarning: invalid value encountered in divide
  cumsum = cumsum/np.max(cumsum)
 75%|███████▌  | 6/8 [00:31<00:10,  5.34s/it]/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:491: RuntimeWarning: invalid value encountered in divide
  cumsum = cumsum/np.max(cumsum)
 88%|████████▊ | 7/8 [00:37<00:05,  5.35s/it]/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:491: RuntimeWarning: invalid value encountered in divide
  cumsum = cumsum/np.max(cumsum)
100%|██████████| 8/8 [00:42<00:00,  5.34s/it]

...DONE...


In [3]:
#################################################
###### MATCH ENSEMBLE ROIS TO MASTER MASK #######
#################################################
#
c.recompute_match_master_mask = False
c.find_matching_cells_to_master_mask()

#
print (c.best_matches)

[[  5 380]
 [  7  15]
 [  7   2]
 [  7   4]]


In [4]:
#######################################################
############# VISUALIZE MATCHED TRACES ################
#######################################################

#
c.y_scale = 1.5
c.show_realtime_bmi_ensembles=True
c.show_upphases=True               # This needs to be fixed/recomputed
c.plotting=False
c.show_rewards = True
c.make_multi_session_traces()


/home/cat/code/bmi/openbmi/analysis/calcium/utils_calcium.py:657: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode1 = scipy.stats.mode(temp).mode


/home/cat/.conda/envs/bmi/lib/python3.8/tkinter/__init__.py:814: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


In [4]:
#######################################################
############# VISUALIZE MATCHED RASTERS ###############
#######################################################

#
c.show_upphases=True                
c.plotting=False
c.show_rewards = True
c.make_multi_session_rasters_allcells()

rasters:  (417, 720000)


loading sessions:   0%|          | 0/8 [00:00<?, ?it/s]

loading sessions: 100%|██████████| 8/8 [00:00<00:00,  9.96it/s]


...ploting...
[ 1500.  4500.  7500. 10500. 13500. 16500. 19500. 22500.]


In [5]:
#######################################################
#######################################################
#######################################################

#
from sklearn.decomposition import PCA

# 
def get_reward_times_all(reward_times):
    r_times = []
    offset=0
    for k in range(len(reward_times)):
        temp = reward_times[k]
        if len(temp)>0:
            r_times.append(temp+offset)

        #
        offset+=90000

    #
    r_times = np.hstack(r_times)

    #
    print ("r_times: ", r_times.shape, r_times[:10])

    #
    return r_times

# remove all zero columns
def remove_zero_frames(X):

    idx = np.where(X.sum(1)>0)[0]
    X = X[idx]
    print ("X: ", X.shape)

    return X

# delete all cells with < 10 bursts
def remove_few_bursts(X):
    diffs = X[1:]-X[:-1]
    print ("diffs: ", diffs.shape)

    # 
    n_bursts = []
    for k in trange(diffs.shape[1]):
        idx = np.where(diffs[:,k]==-1)[0]
        n_bursts.append(idx.shape[0])

    #
    n_bursts = np.array(n_bursts)

    #
    idx = np.where(n_bursts>8*50*2)[0]
    X = X[:, idx]
    print ("X: ", X.shape)

    return X

#
def plot_scatter(X_pca):
    fig=plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_pca[::subsample,0], 
            X_pca[::subsample,1], 
            X_pca[::subsample,2],
            c=clrs[::subsample],
            s=20,
            alpha=.2,
            #edgecolors='black'
            )

    # plot r_times as large red dots
    ax.scatter(X_pca[r_times,0], 
            X_pca[r_times,1], 
            X_pca[r_times,2],
            c='red',
            s=100,
            alpha=.7
            )


    plt.show()
###########################################################
###########################################################
###########################################################
r_times = get_reward_times_all(c.reward_times)


# run PCA on rasters
X = c.rasters.T
print ("X: ", X.shape)

#
pca = PCA()
X_pca = pca.fit_transform(X)

# plot PCA in 3d
subsample = 10
clrs = np.arange(X_pca.shape[0])

#





r_times:  (487,) [ 94927  96368  96502 100392 105912 112595 119548 119852 120359 121199]
X:  (720000, 417)


: 

In [130]:
# # plot variance explained curve
# fig=plt.figure()
# plt.plot(pca.explained_variance_ratio_)

# # same but cumulative
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.show()

In [88]:
###############################################
###############################################
###############################################

# # run PCA just on the neural data of the reward times

# shifts = np.arange(-60,30,3)
# plt.figure()
# randomize = False
# for ctr, shift in enumerate(shifts):
#     ax=plt.subplot(3,10,ctr+1)

#     #
#     plt.title(str(round(shift/30.,2)))
    
#     #
#     X_reward = []
#     for r_time in r_times:

#         if randomize: 
#             idx = np.random.randint(300,3000)
#             temp = X[r_time+shift+idx:r_time+shift+idx+1]
#         else:
#             temp = X[r_time+shift:r_time+shift+1]
        
#         #
#         X_reward.append(temp)

#     #
#     X_reward = np.vstack(X_reward)
#     #
#     pca = PCA()
#     X_pca = pca.fit_transform(X_reward)

#     # plot PCA in 3d
#     subsample = 1
#     clrs = np.arange(X_pca.shape[0])

#     #
#     #ax = fig.add_subplot(111)#, projection='3d')
#     ax.scatter(X_pca[::subsample,0], 
#             X_pca[::subsample,1], 
#             # X_pca[::subsample,2],
#             c=clrs[::subsample],
#             s=100,
#             alpha=.5,
#             #edgecolors='black'
#             )
#     plt.xticks([])
#     plt.yticks([])
#     #
# # plt.xlabel("PC1")
# # plt.ylabel("PC2")

# # #
# # plt.suptitle("PCA on #"+str(X_pca.shape[0])+" reward times\n(blue=early, yellow=late sessions)")

# plt.show()

In [87]:
# # same but using UMAP 
# import umap

# #
# shifts = np.arange(-60,30,3)
# plt.figure()
# ctr=0
# randomize = False
# for shift in tqdm(shifts):
#     ax=plt.subplot(3,10,ctr+1)
    
#     #
#     plt.title(str(round(shift/30.,2)))
    
    
#     #
#     X_reward = []
#     for r_time in r_times:
        
#         if randomize:
#             idx = np.random.randint(0,1000)
#             temp = X[r_time+idx:r_time+idx+1]
#         else:
#             temp = X[r_time+shift:r_time+shift+1]

#         #
#         X_reward.append(temp)

#     #
#     X_reward = np.vstack(X_reward)
#     #print ("X_reward: ", X_reward.shape)
#     #
#     reducer = umap.UMAP()
#     #print ("X_reward: ", X_reward.shape)
#     X_pca = reducer.fit_transform(X_reward)

#     # plot PCA in 3d
#     subsample = 1
#     clrs = np.arange(X_pca.shape[0])

#     #
#     #ax = fig.add_subplot(111)#, projection='3d')
#     ax.scatter(X_pca[::subsample,0], 
#             X_pca[::subsample,1], 
#             # X_pca[::subsample,2],
#             c=clrs[::subsample],
#             s=50,
#             alpha=.5,
#             edgecolors='black'
#             )
    
#     #
#     plt.xticks([])
#     plt.yticks([])

#     #
#     ctr+=1
    

# plt.show()

100%|██████████| 30/30 [01:10<00:00,  2.36s/it]


In [1]:
######################################################################
############# PCA/UMAP ON CHUNKS OF DATA AROUND REWARD TIMES #########
######################################################################
# compute PCA over the entire 2 secs prior to reward

window1 = 5
starts = np.arange(-30,31,window1)

#
r_times_local = r_times.copy()


#
ctr=0
window2 = window1
randomize = True

# delete first 50 rewards
remove_early_sess = False
if remove_early_sess:
    r_times_local = r_times_local[100:]

#
run_pca = False

#
fig = plt.figure()
for start in tqdm(starts):
    #
    ax=plt.subplot(1,13,ctr+1)

    #
    plt.ylabel(str(round(start/30.,2))+" sec")

    #
    X_2sec = []
    for k in range(len(r_times_local)):
        if randomize:
            idx = np.random.randint(100,500)
            temp = X[r_times_local[k]+start+idx:r_times_local[k]+start+idx+window2].flatten()
        else:
            temp = X[r_times_local[k]+start:r_times_local[k]+start+window2].flatten()
        
        #
        X_2sec.append(temp)

    #
    X_2sec = np.vstack(X_2sec)

    # triage outliers using percentiles euclidean distance
    
    
    # now run PCA on this
    if run_pca:
        pca = PCA()
        X_pca = pca.fit_transform(X_2sec)

        # print variance explained
        print ("Explained variance: ", pca.explained_variance_ratio_[:10])
        plt.suptitle("PCA on "+str(round(window2/30.,2))+" sec chunks"+" (randomize="+str(randomize)+")")

    else:
        reducer = umap.UMAP()
        X_pca = reducer.fit_transform(X_2sec)
        plt.suptitle("UMAP on "+str(round(window2/30.,2))+" sec chunks"+" (randomize="+str(randomize)+")")

    # plot PCA in 3d
    subsample = 1
    clrs = np.arange(X_pca.shape[0])

    #
    ax.scatter(X_pca[::subsample,0],
                X_pca[::subsample,1],
                c=clrs[::subsample],
                s=50,
                alpha=.5,
                edgecolors='black'
                )
    #
    plt.xticks([])
    plt.yticks([])


    #
    ctr+=1

#
plt.show()

NameError: name 'np' is not defined

In [131]:
print (c.best_matches)

[[  5 380]
 [  7  15]
 [  7   2]
 [  7   4]]


In [3]:
#############################################
############# PCA/UMAP ON JUST ROIS #########
#############################################
# compute PCA over the entire 2 secs prior to reward
#
window1 = 5
starts = np.arange(-30,31,window1)

#
select_rois = False
rois = c.best_matches[:,1]


#
r_times_local = r_times.copy()

#
ctr=0
window2 = window1
randomize = False

# delete first 50 rewards
remove_early_sess = False
if remove_early_sess:
    r_times_local = r_times_local[100:]

#
run_pca = True

#
fig = plt.figure()
for start in tqdm(starts):
    #
    ax=plt.subplot(1,13,ctr+1)

    #
    plt.ylabel(str(round(start/30.,2))+" sec")

    #
    X_2sec = []
    for k in range(len(r_times_local)):
        if randomize:
            idx = np.random.randint(100,500)
            temp = X[r_times_local[k]+start+idx:r_times_local[k]+start+idx+window2].flatten()
        else:
            temp = X[r_times_local[k]+start:r_times_local[k]+start+window2].flatten()
        
        # 
        if select_rois:
            temp = temp[rois]

        #
        X_2sec.append(temp)

    #
    X_2sec = np.vstack(X_2sec)

    # triage outliers using percentiles euclidean distance
    
    
    # now run PCA on this
    if run_pca:
        pca = PCA()
        X_pca = pca.fit_transform(X_2sec)

        # print variance explained
        print ("Explained variance: ", pca.explained_variance_ratio_[:10])
        plt.suptitle("PCA on "+str(round(window2/30.,3))+" sec chunks"+" (randomize="+str(randomize)+")")

    else:
        reducer = umap.UMAP()
        X_pca = reducer.fit_transform(X_2sec)
        plt.suptitle("UMAP on "+str(round(window2/30.,3))+" sec chunks"+" (randomize="+str(randomize)+")")

    # plot PCA in 3d
    subsample = 1
    clrs = np.arange(X_pca.shape[0])

    #
    ax.scatter(X_pca[::subsample,0],
                X_pca[::subsample,1],
                c=clrs[::subsample],
                s=50,
                alpha=.5,
                edgecolors='black'
                )
    #
    plt.xticks([])
    plt.yticks([])


    #
    ctr+=1

#
plt.show()



AttributeError: 'ProcessCalcium' object has no attribute 'best_matches'

In [216]:
###############################################################################
######### RUN PCA on ROIS only 4D Problem, not super interesting ##############
###############################################################################

# this analysis has no point because the points are actually descrete locations

# #
# select_rois = True
# rois = c.best_matches[:,1]

# #
# r_times_local = r_times.copy()

# #
# ctr=0
# window2 = window1
# randomize = False

# # delete first 50 rewards
# remove_early_sess = False
# if remove_early_sess:
#     r_times_local = r_times_local[100:]

# #
# run_pca = True

# # make PCA object from many random slices of the real data
# idx = np.random.randint(0,X.shape[0],100000)

# # compute all possible boolean combinations of 4 vectors
# X1 = []
# for p in range(100):
#     for k in range(16):
#         X1.append(np.array([int(x) for x in np.binary_repr(k, width=4)])
#                 +np.random.rand(4)*0.001-0.0005)

# #    
# X1 = np.vstack(X1)

# #
# print ("X1: ", X1)

# #
# pca = PCA()
# pca.fit(X1)
# X_pca_background = pca.transform(X1)

# #
# vecs = []
# for k in range(16):
#     vecs.append(np.array([int(x) for x in np.binary_repr(k, width=4)]))
# vecs = np.vstack(vecs)
        
# ###############################################
# fig = plt.figure()
# ax=plt.subplot(111, projection='3d')

# ##
# ax.scatter(X_pca_background[:,0]+np.random.rand(X_pca_background.shape[0])*0.1-0.05, 
#             X_pca_background[:,1]+np.random.rand(X_pca_background.shape[0])*0.1-0.05, 
#             X_pca_background[:,2]+np.random.rand(X_pca_background.shape[0])*0.1-0.05,
#             alpha=1,
#             c='black'
#             )

# # write the vec value as text at each X_pca_background point
# X_vecs = pca.transform(vecs)
# shift2 = 0.005
# for k in range(X_vecs.shape[0]):
#     ax.text(X_vecs[k,0]+shift2, 
#             X_vecs[k,1]+shift2, 
#             X_vecs[k,2]+shift2,
#             str(vecs[k]),
#             size=16,
#             zorder=1,
#             color='k'
#             )

# # loop over each reward and plot it
# start = -60
# end = 0

# # make viridis colormap for n colors
# from matplotlib import cm
# n = len(r_times_local)
# clrs = cm.get_cmap('viridis', n)

# #
# for k in range(len(r_times_local)):

#     #
#     if randomize:
#         idx = np.random.randint(100,500)
#         temp = X[r_times_local[k]+start+idx:r_times_local[k]+end+idx]
#     else:
#         temp = X[r_times_local[k]+start:r_times_local[k]+end]

#     # 
#     if select_rois:
#         temp = temp[:,rois]

#     #
#     X_pca = pca.transform(temp)

#     # plot PCA in 3d
#     # add a bit of jitter to the lines
#     ax.plot(X_pca[:,0] + np.random.rand(X_pca.shape[0])*0.01-0.005,
#             X_pca[:,1] + np.random.rand(X_pca.shape[0])*0.01-0.005,
#             X_pca[:,2] + np.random.rand(X_pca.shape[0])*0.01-0.005,
#             color=plt.cm.viridis(k/n),
#             alpha=.2,
#             )

# # visualize an early trajectry vs a late one
# early_id = 1
# late_id = -1
# early = X[r_times_local[early_id]+start:r_times_local[early_id]+end][:,rois]
# late = X[r_times_local[late_id]+start:r_times_local[late_id]+end][:,rois]

# # run through pca.transform and then plot
# early_pca = pca.transform(early)
# late_pca = pca.transform(late)

# # plot PCA in 3d
# ax.plot(early_pca[:,0],
#         early_pca[:,1],
#         early_pca[:,2],
#         c='black',
#         linewidth=5,
#     )

# # same but scatter plot
# ax.scatter(early_pca[:,0],
#         early_pca[:,1],
#         early_pca[:,2],
#         c='black',
#         linewidth=5,
# )

# ax.plot(late_pca[:,0],
#         late_pca[:,1],
#         late_pca[:,2],
#         c='red',
#         linewidth=5,
# )

# # same but scatter plot
# ax.scatter(late_pca[:,0],
#         late_pca[:,1],
#         late_pca[:,2],
#         c='red',
#         linewidth=5,
# )



# # 
# plt.suptitle("Time window "+str(round(start/30.,2))+" to "+str(round(end/30.,2))+" sec"+
#              "\n(roi1, roi2, roi3, roi4)")

# plt.show()

X1:  [[ 1.89668908e-04  2.85031882e-04 -9.77818450e-05 -4.50158185e-05]
 [ 1.62704051e-04 -9.97856527e-05  3.49732664e-05  9.99908014e-01]
 [ 3.06477731e-04  6.70330010e-05  9.99910798e-01 -2.60284392e-04]
 ...
 [ 1.00010959e+00  1.00049536e+00 -4.34840390e-05  9.99566421e-01]
 [ 9.99808644e-01  1.00027670e+00  9.99780239e-01 -3.85187774e-04]
 [ 1.00047889e+00  1.00041298e+00  9.99796924e-01  1.00041248e+00]]


: 